<a href="https://colab.research.google.com/github/EdwarMontano/Trainer_RecognizeMedications/blob/main/Remote_Trainer_RecognizeMedications.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table>
<tbody>
<tr>
<td><img width="100px" src="https://upload.wikimedia.org/wikipedia/commons/e/eb/Univalle.svg" alt="Univalle"> </td>
<td><img width="220px" src="https://i.ibb.co/6vdWxb4/PSI-LOGO.png" alt="PSI"></td>
</tr>
</tbody>
</table>

# Como Entrenar un modelo con de reconocimiento de medicamentos con  YOLOv8 con etiquetaas cuadro delimintadores orientados

---
[![GitHub](https://badges.aleen42.com/src/github.svg)](https://github.com/EdwarMontano/Trainer_RecognizeMedications)

## Consejo: Utiliza la aceleración por GPU

Si está ejecutando este cuaderno en Google Colab, vaya a `Editar` -> `Configuraciones de Notebook ` -> `Aceleración por Hardware `, configurarlo en `GPU`, y luego hacer clic en `Guardar`. Esto garantizará que tu cuaderno utilice una GPU, lo que acelerará significativamente los tiempos de entrenamiento del modelo.

## Pasos de este tutorial

En este tutorial, vamos a cubrir:

- Verificar que nuestro entorno de Colab tenga una GPU
- Instalar YOLOv8
- Preparar un conjunto de datos
- Entrenar un modelo YOLOv8 OBB
- Ejecutar la inferencia en nuestro modelo

¡Sin más preámbulos, comencemos!

## Antes de empezar

Asegurémonos de que tenemos acceso a la GPU. Para ello podemos usar el comando `nvidia-smi`. En caso de cualquier problema navegue a `Edit` -> `Notebook settings` -> `Hardware accelerator`, póngalo en `GPU`, y pulse `Save`.

In [ ]:
!nvidia-smi

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

## Install YOLOv8

To install YOL0v8, run the following command:

In [ ]:
!pip install ultralytics -q
!pip install optuna -q
!pip install roboflow --quiet

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
import ultralytics
ultralytics.checks()

## Preparar un conjunto de datos personalizado

Construir un conjunto de datos personalizado puede ser un proceso tedioso. Puede tomar docenas o incluso cientos de horas recolectar imágenes, etiquetarlas y exportarlas en el formato adecuado. Afortunadamente, Roboflow hace que este proceso sea lo más sencillo y rápido posible. ¡Déjanos mostrarte cómo!


### Paso 1: Crear un proyecto

Antes de comenzar, necesitas crear una [cuenta](https://app.roboflow.com/login) en Roboflow. Una vez que lo hagas, puedes crear un nuevo proyecto en el [panel de control](https://app.roboflow.com/) de Roboflow. Elige "Detección de Objetos" como el tipo de proyecto.

<img src="https://github.com/EdwarMontano/Trainer_RecognizeMedications/assets/52388306/3afa0b1b-5629-491f-b145-9f7ba9a77325" alt="Detección de objetos seleccionada en el cuadro de diálogo Crear Proyecto de Roboflow" height="300" />

### Paso 2: Subir imágenes

A continuación, añade los datos a tu proyecto recién creado. Puedes hacerlo a través de la API o mediante nuestra [interfaz web](https://docs.roboflow.com/adding-data/object-detection).

Si arrastras y sueltas un directorio con un conjunto de datos en un formato compatible, el panel de control de Roboflow leerá automáticamente las imágenes y anotaciones juntos.

<img src="https://github.com/EdwarMontano/Trainer_RecognizeMedications/assets/52388306/40dd4aec-8756-4852-bcb9-cfc316bb1e3a" alt="Subiendo imágenes a Roboflow" height="300" />

### Paso 3: Etiquetar datos

Si solo tienes imágenes, puedes etiquetarlas con cuadros delimitadores orientados en [Roboflow Annotate](https://docs.roboflow.com/annotate).

**Para etiquetar un cuadro delimitador orientado, usa nuestra herramienta de anotación de polígonos.**

También puedes tomar un conjunto de datos de segmentación existente de tu espacio de trabajo o de Roboflow Universe y exportarlo como OBB.

<img src="https://github.com/EdwarMontano/Trainer_RecognizeMedications/assets/52388306/9b0e7c1e-b71f-4d71-ac3a-6d4b0261d8d3" alt="Anotar una imagen" height="300" />

### Paso 4: Generar una nueva versión del conjunto de datos

Ahora que tenemos nuestras imágenes y anotaciones añadidas, podemos generar una versión del conjunto de datos. Al generar una versión, puedes optar por añadir preprocesamiento y aumentos. Este paso es completamente opcional, sin embargo, puede permitirte mejorar significativamente la robustez de tu modelo.

<img src="https://github.com/EdwarMontano/Trainer_RecognizeMedications/assets/52388306/9a88edc9-6e0f-48c6-9f53-5683d846bb55" alt="Generar una versión del conjunto de datos" height="300" />

### Paso 5: Exportar conjunto de datos

Una vez que se genera la versión del conjunto de datos, podemos descargarla para usarla en el entrenamiento de un modelo.

![Generar una versión del conjunto de datos](https://media.roboflow.com/keypoint/export.png)

## Cargar Dataset

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets


import roboflow

roboflow.login()

rf = roboflow.Roboflow()

project = rf.workspace("uv-ipsi6").project("recognice-medications-boxes")
dataset = project.version(5).download("yolov8-obb")

import yaml

with open(f'{dataset.location}/data.yaml', 'r') as file:
    data = yaml.safe_load(file)

data['path'] = dataset.location

with open(f'{dataset.location}/data.yaml', 'w') as file:
    yaml.dump(data, file, sort_keys=False)

In [ ]:
model = YOLO("yolov8n.pt")

# Tune hyperparameters on COCO8 for 30 epochs
model.tune(data="coco8.yaml", epochs=2, iterations=10, optimizer="AdamW", plots=False, save=False, val=False)


In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)

# Train the model with 2 GPUs
results = model.train(data='/content/dataset/data.yaml', epochs=200, imgsz=640, device=0)

In [ ]:
%tensorboard --logdir logs

In [ ]:
# Validate model

from ultralytics import YOLO

# Load a model
model = YOLO('runs/detect/train/weights/best.pt')  # load a custom model

# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category

In [ ]:
#  Prediction using trained model
import os
from ultralytics import YOLO

image_dir = '/content/dataset/test/images'

# Check if the directory exists
if os.path.exists(image_dir):
    # Run inference if the directory exists
    model.predict(image_dir, save=True, imgsz=640, conf=0.2)
else:
    print(f"Error: The specified path '{image_dir}' does not exist.")

# Load a pretrained YOLOv8n model
model = YOLO('runs/detect/train/weights/best.pt')

# Run inference
model.predict('/content/dataset/test/images', save=True, imgsz=640, conf=0.2)

In [ ]:
#  Export model to tflite


from ultralytics import YOLO

# Load a model
model = YOLO('runs/detect/train/weights/best.pt')  # load a custom trained model

# Export the model
model.export(format='tflite')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Prediction using custom tflite model

#  Prediction using trained model

from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('runs\\detect\\train3\\weights\\best_saved_model\\best_float32.tflite')

# Run inference
model.predict('test_images', save=True, imgsz=640, conf=0.2)

In [ ]:
!python --version